In [1]:
import cv2 
import math
import numpy as np
import imutils
from numba import jit, cuda
from timeit import default_timer as timer
import math as m
from PIL import Image 
from IPython.display import clear_output

In [1]:
import cv2 
import math
import numpy as np
import imutils
from numba import jit, cuda
from timeit import default_timer as timer
import math as m
from PIL import Image 
from IPython.display import clear_output

def Mrotate(rotateImage, angle):
    imgHeight, imgWidth = rotateImage.shape[0], rotateImage.shape[1]
    centreY, centreX = imgHeight//2, imgWidth//2
    rotationMatrix = cv2.getRotationMatrix2D((centreY, centreX), angle, 1.0)
    cosofRotationMatrix = np.abs(rotationMatrix[0][0])
    sinofRotationMatrix = np.abs(rotationMatrix[0][1])
    newImageHeight = int((imgHeight * sinofRotationMatrix) + (imgWidth * cosofRotationMatrix))
    newImageWidth = int((imgHeight * cosofRotationMatrix) +  (imgWidth * sinofRotationMatrix))
    rotationMatrix[0][2] += (newImageWidth/2) - centreX
    rotationMatrix[1][2] += (newImageHeight/2) - centreY
    rotatingimage = cv2.warpAffine(rotateImage, rotationMatrix, (newImageWidth, newImageHeight))
    return rotatingimage

def calculate_ssim(img1, img2):
    if(len(img1.shape)<3):
        if ((img1.shape[0])<(img2.shape[0])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1]=img1
            img1=img3
            
        elif ((img1.shape[0])>(img2.shape[0])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1]=img2
            img2=img3
            
        if ((img1.shape[1])<(img2.shape[1])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1]=img1
            img1=img3
            
        elif ((img1.shape[1])>(img2.shape[1])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1]=img2
            img2=img3
    
    elif(len(img1.shape)==3):
        if ((img1.shape[0])<(img2.shape[0])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img1
            img1=img3
            
        elif ((img1.shape[0])>(img2.shape[0])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img2
            img2=img3
            
        if ((img1.shape[1])<(img2.shape[1])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img1
            img1=img3
            
        elif ((img1.shape[1])>(img2.shape[1])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img2
            img2=img3
            
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')
        
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    if(len(img1.shape)<3):
        if ((img1.shape[0])<(img2.shape[0])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1]=img1
            img1=img3
            
        if ((img1.shape[0])>(img2.shape[0])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1]=img2
            img2=img3
            
        if ((img1.shape[1])<(img2.shape[1])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1]=img1
            img1=img3
            
        if ((img1.shape[1])>(img2.shape[1])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1]=img2
            img2=img3
    
    elif(len(img1.shape)==3):
        if ((img1.shape[0])<(img2.shape[0])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img1
            img1=img3
            
        if ((img1.shape[0])>(img2.shape[0])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img2
            img2=img3
            
        if ((img1.shape[1])<(img2.shape[1])):
            X=img1.shape[0]
            Y=img2.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img1.shape[0]/2))
            x1=int((X/2)+(img1.shape[0]/2))
            y0=int((Y/2)-(img1.shape[1]/2))
            y1=int((Y/2)+(img1.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img1
            img1=img3
            
        if ((img1.shape[1])>(img2.shape[1])):
            X=img2.shape[0]
            Y=img1.shape[1]
            img3 = np.full((X,Y,3), 0, dtype=np.uint8)
            x0=int((X/2)-(img2.shape[0]/2))
            x1=int((X/2)+(img2.shape[0]/2))
            y0=int((Y/2)-(img2.shape[1]/2))
            y1=int((Y/2)+(img2.shape[1]/2))
            img3[x0:x1,y0:y1,:]=img2
            img2=img3
            
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

def ssim1(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2
    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *(sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

@jit(target_backend='cuda')
def crop(img):
    mx_up=img.shape[0]
    mx_lt=img.shape[1]
    mx_rt=0
    mx_dn=0
    for j in range (img.shape[0]):
        f=0
        for k in range (img.shape[1]):
            if(img[j,k]!=0):
                img[j,k]=255
                f+=1
                if (f==1 and k<mx_lt):
                    mx_lt=k
                if (f>=1 and k>mx_rt):
                    mx_rt=k
    for j in range (img.shape[1]):
        f=0
        for k in range (img.shape[0]):
            if(img[k,j]!=0):
                img[k,j]=255
                f+=1
                if (f==1 and k<mx_up):
                    mx_up=k
                if (f>=1 and k>mx_dn):
                    mx_dn=k
    mx_lt-=1#9
    mx_dn+=2#10
    mx_rt+=2#10
    mx_up-=1#9
    return img[mx_up:mx_dn,mx_lt:mx_rt]

def find_angle(a,b):
    return (m.atan2(a[1] - b[1], a[0] - b[0]) * 180 / m.pi)-90

def distance(p1,p2):
    return (m.sqrt((p1[0] - p2[0])**2+(p1[1] - p2[1])**2))

def detect_contour(c):
    mxl=0
    mxr=0
    mnl=10**5
    mnr=10**5
    for i in c:
        if(i[0][0]<mnl):
            mnl=i[0][0]
            mnlc=i[0][1]
        if(i[0][0]>mxl):
            mxl=i[0][0]
            mxlc=i[0][1]
        if(i[0][1]<mnr):
            mnr=i[0][1]
            mnrc=i[0][0]
        if(i[0][1]>mxr):
            mxr=i[0][1]
            mxrc=i[0][0]
    return ((mxr-mnr)*(mxl-mnl),((mxrc,mxr),(mnl,mnlc),(mnrc,mnr),(mxl,mxlc)))

def rotate(image,a,s=1.1):
    if(len(image.shape)<3):
        img1=image
        x,y=img1.shape
        X,Y=int(2.5*x),int(2.5*y)
        image = np.full((X,Y), 0, dtype=np.uint8)
        x0=int((X/2)-(img1.shape[0]/2))
        x1=int((X/2)+(img1.shape[0]/2))
        y0=int((Y/2)-(img1.shape[1]/2))
        y1=int((Y/2)+(img1.shape[1]/2))
        image[x0:x1,y0:y1]=img1

        height, width = image.shape[:2]
        center = (width/2, height/2)
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=a, scale=s)
        return cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))
    
    elif(len(image.shape)==3):
        img1=image
        x,y,z=img1.shape
        X,Y=int(2*x),int(2*y)
        image = np.full((X,Y,z), 0, dtype=np.uint8)
        x0=int((X/2)-(img1.shape[0]/2))
        x1=int((X/2)+(img1.shape[0]/2))
        y0=int((Y/2)-(img1.shape[1]/2))
        y1=int((Y/2)+(img1.shape[1]/2))
        image[x0:x1,y0:y1,:]=img1

        height, width = image.shape[:2]
        center = (width/2, height/2)
        rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=a, scale=s)
        return cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))

def edges(image):
    med_val = np.median(image)
    # blurred_img = cv2.blur(image,ksize=(3,3))
    blurred_img = image
    thr=1
    lower = int(max(0, (thr//10)* med_val))
    upper = int(min(255,(2-(thr//10))* med_val))
    return cv2.Canny(image=blurred_img, threshold1=lower , threshold2=upper)

def contour(img):
    img=edges(img)
    ret, img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
    cnts = imutils.grab_contours(cv2.findContours(img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE))
    mxa=-1
    x,y = img.shape[:2]
    external_contours = np.zeros((x,y))
    for c in cnts:
        M = cv2.moments(c)
        if M["m00"] != 0:
            cv2.drawContours(external_contours, [c], -1, (255, 255, 255), 1)
            ar,crd = detect_contour(c)
            if(ar>=mxa):
                ct=c
                mxa=ar
                crdf=crd
    external_contours_c = np.zeros((x,y))
    if(mxa!=-1):
        cv2.drawContours(external_contours_c, [ct], -1, (255, 255, 255), 1)
        return(external_contours,external_contours_c,crdf)
    else:
        crdf=0
        return(external_contours,external_contours_c,crdf)
    
def len_cnts(img):
    ret, img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
    cnts = imutils.grab_contours(cv2.findContours(img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE))
    return len(cnts)
def ssim(idl,img):
    ssim=1
    
    c11=.07
    c12=.19
    c13=.09
    c14=.28
    
    c21=.81
    c22=.93
    c23=.72
    c24=.91
    
    img1 =img[int(c11*img.shape[0]):int(c12*img.shape[0]),int(c13*img.shape[1]):int(c14*img.shape[1])]#1
    img2 =img[int(c21*img.shape[0]):int(c22*img.shape[0]),int(c23*img.shape[1]):int(c24*img.shape[1])]#3
    
    # img1 =img[int(.07*img.shape[0]):int(.19*img.shape[0]),int(.06*img.shape[1]):int(.26*img.shape[1])]#1
    # img2 =img[int(.85*img.shape[0]):int(.95*img.shape[0]),int(.74*img.shape[1]):int(.95*img.shape[1])]#3
    
    img3 =img[int(c11*img.shape[0]):int(c12*img.shape[0]),int(c23*img.shape[1]):int(c24*img.shape[1])]#4
    img4 =img[int(c21*img.shape[0]):int(c22*img.shape[0]),int(c13*img.shape[1]):int(c14*img.shape[1])]#2
    
    img12=img[int(c12*img.shape[0]):int(c21*img.shape[0]),int(c13*img.shape[1]):int(c14*img.shape[1])]
    img23=img[int(c21*img.shape[0]):int(c22*img.shape[0]),int(c14*img.shape[1]):int(c23*img.shape[1])]
    img34=img[int(c12*img.shape[0]):int(c21*img.shape[0]),int(c23*img.shape[1]):int(c24*img.shape[1])]
    img41=img[int(c11*img.shape[0]):int(c12*img.shape[0]),int(c14*img.shape[1]):int(c23*img.shape[1])]
    img55=img[int(c12*img.shape[0]):int(c21*img.shape[0]),int(c14*img.shape[1]):int(c23*img.shape[1])]
    
    if not(len_cnts(img1)!=0 and len_cnts(img2)!=0 and len_cnts(img3)!=0 and len_cnts(img4)!=0 and len_cnts(img12)==0 and len_cnts(img23)==0 and len_cnts(img34)==0 and len_cnts(img41)==0 and len_cnts(img55)==0):
        ssim=-1000
    imgt = np.zeros((img.shape[0],img.shape[1]))
    
    imge = cv2.rectangle(img , (int(c13*img.shape[1]),int(c11*img.shape[0])), (int(c14*img.shape[1]),int(c12*img.shape[0])), 255, 1)
    imge = cv2.rectangle(imge, (int(c23*img.shape[1]),int(c21*img.shape[0])), (int(c24*img.shape[1]),int(c22*img.shape[0])), 255, 1)
    
    imge = cv2.rectangle(imge, (int(c23*img.shape[1]),int(c11*img.shape[0])), (int(c24*img.shape[1]),int(c12*img.shape[0])), 255, 1)
    imge = cv2.rectangle(imge, (int(c11*img.shape[1]),int(c21*img.shape[0])), (int(c14*img.shape[1]),int(c22*img.shape[0])), 255, 1)
    
    # imge = cv2.rectangle(imge, (int(c21*img.shape[1]),int(c11*img.shape[0])), (int(c24*img.shape[1]),int(c12*img.shape[0])), 255, 1)
    # imge = cv2.rectangle(imge, (int(c11*img.shape[1]),int(c21*img.shape[0])), (int(c14*img.shape[1]),int(c22*img.shape[0])), 255, 1)
    
    # img = cv2.circle(img,(int(.06*img.shape[0]),int(.06*img.shape[1])), 1, 255, 3)
    # img = cv2.circle(img,(int(.96*img.shape[1]),int(.96*img.shape[0])), 3, 255, 5)
    # print(img.shape)
    
    # img[int(.2*img.shape[0]):int(.8*img.shape[0]),int(.2*img.shape[1]):int(.8*img.shape[1])]=imgt[int(.2*img.shape[0]):int(.8*img.shape[0]),int(.2*img.shape[1]):int(.8*img.shape[1])].copy()
    # idl[int(.2*img.shape[0]):int(.8*img.shape[0]),int(.2*img.shape[1]):int(.8*img.shape[1])]=imgt[int(.2*img.shape[0]):int(.8*img.shape[0]),int(.2*img.shape[1]):int(.8*img.shape[1])].copy()
    # return (ssim1(idl,img)+ssim),img12,img23,img34,img41,img1,img4,img2,img3
    return (ssim1(idl,img)+ssim),imge

    
def find_cntr(name):
    start = timer()
    
    # cap = cv2.VideoCapture(int(cam),cv2.CAP_DSHOW)
    # while(True):
    # ret, img = cap.read()
    # cv2.imwrite('snapshot.png', img)
    path='datasets/Train/Non-defective/non-defective_'+str(name)+'.jpg'
    img = cv2.imread(path)
    idl = cv2.imread('dataset_new/IDEAL1.jpg',0)

    # print(timer()-start,1)

    im,lim,p=contour(img)

    # imt=im[p[2][1]-10:p[0][1]+10,p[1][0]-10:p[3][0]+10]
    imt=im
    # print(timer()-start,2)

    # im = cv2.circle(im, p[0], 3, 255, 3)
    # im = cv2.circle(im, p[1], 3, 255, 3)
    # im = cv2.circle(im, p[2], 3, 255, 3)
    d1=distance(p[0],p[1])
    d2=distance(p[1],p[2])
    d3=distance(p[0],p[2])
    ta=[d1,d2,d3]
    ta.sort()
    # print(d1,d2,d3)
    if(d2==ta[1]):
        ang=find_angle(p[1],p[2])
        im = cv2.circle(lim, p[2], 3, 255, 3)
        im = cv2.circle(lim, p[1], 3, 255, 3)
    elif(d1==ta[1]):
        ang=find_angle(p[1],p[0])
        im = cv2.circle(lim, p[0], 3, 255, 3)
        im = cv2.circle(lim, p[1], 3, 255, 3)
    elif(d3==ta[1]):
        ang=find_angle(p[2],p[0])
        im = cv2.circle(lim, p[2], 3, 255, 3)
        im = cv2.circle(lim, p[2], 3, 255, 3)

    # print(timer()-start,3)

    # cv2.imwrite('im.png',im)
    # imag = Image.open(r"im.png")
    # rim= np.array(imag.rotate(ang))
    # rim = imutils.rotate(im, angle=ang)

    # print(timer()-start,4)

    mxa=10**9
    # for i in range(-5,5,1):
    for i in range(-1,3,1):
        rimt = rotate(imt,ang+i)
        cimt = crop(rimt)
        if(cimt.shape[0]*cimt.shape[1]<mxa):
            mxa=cimt.shape[0]*cimt.shape[1]
            an = i
            cim=cimt.copy()
            rim=rimt.copy()

    # print(timer()-start,5,'area -',an)
    # print('area -',an,end=' - ')

    # sim,img12,img23,img34,img41,img1,img2,img3,img4=calculate_ssim(idl,cim)
    sim,fimg=calculate_ssim(idl,cim)


    # print(timer()-start,6)
    # print(sim,end='')
    if(sim<0.8):
        print("Defective",sim,end='')
    else:
        print("Defect-free",sim,end='')



    cv2.namedWindow("original", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("original", 330, 255)
    cv2.namedWindow("processed1", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("processed1", 330, 255)
    cv2.namedWindow("processed2", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("processed2", 330, 255)
    cv2.namedWindow("processed3", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("processed3", 330, 255)
    cv2.namedWindow("processed4", cv2.WINDOW_NORMAL)        
    cv2.resizeWindow("processed4", 330, 255)

    cv2.imshow('original', img)
    cv2.imshow('processed1', lim)
    cv2.imshow('processed2', imt)
    cv2.imshow('processed3', rim)
    cv2.imshow('processed4', fimg)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

    cv2.waitKey(0)
    # cap.release()
    cv2.destroyAllWindows()
    print(' - ',timer()-start)

In [2]:
find_cntr(182)

Defective -999.092248333894 -  25.794603800000004


In [4]:
# start1 = timer()
# for i in range(0,199):
#     if(i not in (-1,117,118)):
#         print(i,end=' - ')
#         find_cntr(i)
# print(timer()-start1)

In [5]:
from IPython.display import clear_output
def find_cntr_c(cam=0):
    
    cap = cv2.VideoCapture(int(cam),cv2.CAP_DSHOW)
    while(True):
        ret, img = cap.read()
        img=img[20:,30:-13]
        cv2.imwrite('snapshot.png', img)
        # path='dataset_new/Training/Defect-free/DF_'+str(name)+'.jpg'
        # img = cv2.imread(path)
        idl = cv2.imread('dataset_new/IDEAL1.jpg',0)

        # print(timer()-start,1)

        im,lim,p=contour(img)
        print(p)
        if(p!=0):
            # imt=im[p[2][1]-30:p[0][1]+30,p[1][0]-30:p[3][0]+30]
            imt=im

            # print(timer()-start,2)

            # im = cv2.circle(im, p[0], 3, 255, 3)
            # im = cv2.circle(im, p[1], 3, 255, 3)
            # im = cv2.circle(im, p[2], 3, 255, 3)
            d1=distance(p[0],p[1])
            d2=distance(p[1],p[2])
            d3=distance(p[0],p[2])
            ta=[d1,d2,d3]
            ta.sort()
            # print(d1,d2,d3)
            if(d2==ta[1]):
                ang=find_angle(p[1],p[2])
                im = cv2.circle(lim, p[2], 3, 255, 3)
                im = cv2.circle(lim, p[1], 3, 255, 3)
            elif(d1==ta[1]):
                ang=find_angle(p[1],p[0])
                im = cv2.circle(lim, p[0], 3, 255, 3)
                im = cv2.circle(lim, p[1], 3, 255, 3)
            elif(d3==ta[1]):
                ang=find_angle(p[2],p[0])
                im = cv2.circle(lim, p[2], 3, 255, 3)
                im = cv2.circle(lim, p[2], 3, 255, 3)

            # print(timer()-start,3)

            # cv2.imwrite('im.png',im)
            # imag = Image.open(r"im.png")
            # rim= np.array(imag.rotate(ang))
            # rim = imutils.rotate(im, angle=ang)

            # print(timer()-start,4)

            mxa=10**9
            # for i in range(-5,5,1):
            for i in range(-1,3,1):
                rimt = rotate(imt,ang+i)
                cimt = crop(rimt)
                if(cimt.shape[0]*cimt.shape[1]<mxa):
                    mxa=cimt.shape[0]*cimt.shape[1]
                    an = i
                    cim=cimt
                    rim=rimt

            # print(timer()-start,5,'area -',an)
            # print('area -',an,end=' - ')

            # sim,img12,img23,img34,img41,img1,img2,img3,img4=calculate_ssim(idl,cim)
            sim,fimg=calculate_ssim(idl,cim)


            # print(timer()-start,6)
            # print(sim,end='')
            if(sim<0):
                print("Defective",sim,end='')
            else:
                print("Defect-free",sim,end='')



            cv2.namedWindow("original", cv2.WINDOW_NORMAL)
            cv2.resizeWindow("original", 330, 255)
            cv2.namedWindow("processed1", cv2.WINDOW_NORMAL)        
            cv2.resizeWindow("processed1", 330, 255)
            cv2.namedWindow("processed2", cv2.WINDOW_NORMAL)        
            cv2.resizeWindow("processed2", 330, 255)
            cv2.namedWindow("processed3", cv2.WINDOW_NORMAL)        
            cv2.resizeWindow("processed3", 330, 255)
            cv2.namedWindow("processed4", cv2.WINDOW_NORMAL)        
            cv2.resizeWindow("processed4", 330, 255)

            cv2.imshow('original', img)
            cv2.imshow('processed1', imt)
            cv2.imshow('processed2', lim)
            cv2.imshow('processed3', rim)
            cv2.imshow('processed4', fimg)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        clear_output(wait=True)

        # cv2.waitKey(0)
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
find_cntr_c(0)

((284, 404), (191, 374), (200, 347), (293, 378))
Defective -999.2669726119632

In [21]:
# import cv2
# cam = cv2.VideoCapture(1,cv2.CAP_DSHOW)
# cam.set(cv2.CAP_PROP_AUTOFOCUS, 0)

# def edges(image):
#     med_val = np.median(image)
#     # blurred_img = image
#     blurred_img = cv2.blur(image,ksize=(11,11))
#     thr=9
#     lower = int(max(0, (thr//10)* med_val))
#     upper = int(min(255,(2-(thr//10))* med_val))
#     return cv2.Canny(image=blurred_img, threshold1=lower , threshold2=upper)

# def nothing(x):
#     pass

# #       key value
# cam.set(3 , 640 ) # width        
# cam.set(4 , 480  ) # height       
# cam.set(10, 200  ) # brightness     min: 0   , max: 255 , increment:1  
# cam.set(11, 200   ) # contrast       min: 0   , max: 255 , increment:1     
# cam.set(12, 200   ) # saturation     min: 0   , max: 255 , increment:1
# cam.set(13, 13  ) # hue         
# cam.set(14, 50   ) # gain           min: 0   , max: 127 , increment:1
# cam.set(15, -3   ) # exposure       min: -7  , max: -1  , increment:1
# cam.set(17, 5000 ) # white_balance  min: 4000, max: 7000, increment:1
# cam.set(28, 8 ) # focus          min: 0   , max: 255 , increment:5

# cv2.namedWindow("frame", cv2.WINDOW_NORMAL)        
# cv2.resizeWindow("frame", 630, 455)

# cv2.namedWindow("frame1", cv2.WINDOW_NORMAL)        
# cv2.resizeWindow("frame1", 630, 455)

# cv2.namedWindow('image', cv2.WINDOW_NORMAL)        
# cv2.resizeWindow('image', 1000, 680)
# cv2.createTrackbar('brightness','image',0,255,nothing)
# cv2.createTrackbar('contrast','image',0,255,nothing)
# cv2.createTrackbar('saturation','image',0,255,nothing)
# cv2.createTrackbar('hue','image',0,255,nothing)
# cv2.createTrackbar('gain','image',0,127,nothing)
# cv2.createTrackbar('exposure','image',0,6,nothing)
# cv2.createTrackbar('white_balance','image',4000,7000,nothing)
# cv2.createTrackbar('focus','image',0,255,nothing)

# while(True):
#     ret, frame = cam.read()
#     cv2.imshow('frame', frame)
#     im=edges(frame)
#     cv2.imshow('frame1', im)
    
#     bri = cv2.getTrackbarPos('brightness','image')
#     con = cv2.getTrackbarPos('contrast','image')
#     sat = cv2.getTrackbarPos('saturation','image')
#     hue = cv2.getTrackbarPos('hue','image')
#     gai = cv2.getTrackbarPos('gain','image')
#     exp = cv2.getTrackbarPos('exposure','image')
#     whi = cv2.getTrackbarPos('white_balance','image')
#     foc = cv2.getTrackbarPos('focus','image')
    
#     cam.set(10, bri) # brightness     min: 0   , max: 255 , increment:1  
#     cam.set(11, con) # contrast       min: 0   , max: 255 , increment:1   
#     cam.set(12, sat) # saturation     min: 0   , max: 255 , increment:1
#     cam.set(13, hue) # hue
#     cam.set(14, gai) # gain           min: 0   , max: 127 , increment:1
#     cam.set(15, exp-7) # exposure       min: -7  , max: -1  , increment:1
#     cam.set(17, whi) # white_balance  min: 4000, max: 7000, increment:1
#     cam.set(28, foc) # focus          min: 0   , max: 255 , increment:5
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
    
# # cv2.waitKey(0)
# cam.release()
# cv2.destroyAllWindows()
# print(frame.shape)

In [7]:
import cv2
import numpy as np
print(0)
# cam = cv2.VideoCapture(0)
cam = cv2.VideoCapture(0,cv2.CAP_DSHOW)
cam.set(3 , 1280 ) # width        
cam.set(4 , 720  ) # height    
print(1)
def edges(image):
    med_val = np.median(image)
    # blurred_img = image
    blurred_img = cv2.blur(image,ksize=(2,2))
    thr=9
    lower = int(max(0, (thr//10)* med_val))
    upper = int(min(255,(2-(thr//10))* med_val))
    # return cv2.Canny(image=blurred_img, threshold1=lower , threshold2=upper)
    return cv2.Canny(image=blurred_img, threshold1=30, threshold2=150)
    
cv2.namedWindow("frame", cv2.WINDOW_NORMAL)        
cv2.resizeWindow("frame", 630, 455)

cv2.namedWindow("frame1", cv2.WINDOW_NORMAL)        
cv2.resizeWindow("frame1", 630, 455)
print(2)
while(True):
    # print(3)
    ret, frame = cam.read()
    # print(4)
    # frame=frame[100:-100,100:-100]
    cv2.imshow('frame', frame)
    # im=edges(frame)
    im,lim,p=contour(frame)
    print(p)
    cv2.imshow('frame1', im)
    cv2.imwrite('edge.jpg',im)
    cv2.imwrite('piece.jpg',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    clear_output(wait=True)
    
# cv2.waitKey(0)
cam.release()
cv2.destroyAllWindows()
print(frame.shape)

0
(720, 1280, 3)
